In [10]:
!pip install anthropic
!pip install openai
!pip install faiss-cpu
!pip install tiktoken

In [10]:
import pandas as pd #library for dataframes
import tiktoken #library to estimate tokens used for each character

import faiss
# a library that allows developers to quickly search for embeddings of
# multimedia documents that are similar to each other. Can use any kind of Vector DB
# only used for demo purposes

from anthropic import Anthropic
from openai import OpenAI

import numpy as np


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
#never share your keys publicly. always load them using .env files. Directly assignment here is for demo purposes ONLY
ANTHROPIC_API_KEY = "XXX"
client_ant = Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

OPENAI_API_KEY = "XXXXX"
client_openai = OpenAI(api_key=OPENAI_API_KEY)

In [14]:
embedding_encoding = "cl100k_base"
max_tokens = 5000  # the maximum for text-embedding-3-small is 8191

# load & inspect dataset
path = "/content/drive/My Drive/Colab Notebooks/AISC/Reviews.csv" #adjust to your own directory
reviews_df = pd.read_csv(path, index_col=0)

encoding = tiktoken.get_encoding(embedding_encoding) #get token usage estimate
top_n = 50 #Get 50 most recent reviews

reviews_df = reviews_df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
reviews_df = reviews_df.dropna()
reviews_df = reviews_df[:top_n]
reviews_df["combined"] = (
    "Title: " + reviews_df.Summary.str.strip() + "; Content: " + reviews_df.Text.str.strip()
)

# omit reviews that are too long to embed - OPTIONAL
reviews_df["n_tokens"] = reviews_df.combined.apply(lambda x: len(encoding.encode(x)))
reviews_df = reviews_df[reviews_df.n_tokens <= max_tokens]

In [15]:
print(reviews_df.shape)
reviews_df.head()

(50, 8)


,Time,ProductId,UserId,Score,Summary,Text,combined,n_tokens
Id,,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...,64
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...,51
3,1219017600,B000LQOCH0,ABXLMWJIXXAIN,4,"""Delight"" says it all",This is a confection that has been around a fe...,"Title: ""Delight"" says it all; Content: This is...",137
4,1307923200,B000UA0QIQ,A395BORC6FGVXV,2,Cough Medicine,If you are looking for the secret ingredient i...,Title: Cough Medicine; Content: If you are loo...,59
5,1350777600,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy,Great taffy at a great price. There was a wid...,Title: Great taffy; Content: Great taffy at a ...,50


In [16]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# CLAUDE does not offer its own embedding model: https://docs.anthropic.com/en/docs/build-with-claude/embeddings
# Generate embeddings using OpenAI
def get_embedding(text, model="text-embedding-3-small"):
    embeddings = client_openai.embeddings.create(
        input = [text],
        model=model
    )
    return embeddings.data[0].embedding

reviews_df['ada_embedding'] = reviews_df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

In [17]:
reviews_df.head()

,Time,ProductId,UserId,Score,Summary,Text,combined,n_tokens,ada_embedding
Id,,,,,,,,,
568405,1350172800,B005OTVL8C,A1UJMVE0LCOI45,5,great flavor boost,If you want to add a hint of Meyer lemon flavo...,Title: great flavor boost; Content: If you wan...,86,"[0.025893114507198334, 0.0037613348104059696, ..."
568406,1274918400,B000P56I7Y,A2ZKNGL20FBK2I,5,Arrived Fine,The product arrived pretty quickly and was eas...,Title: Arrived Fine; Content: The product arri...,42,"[-0.004842398688197136, 0.0034032922703772783,..."
568407,1336867200,B0039KE8Y2,A392FNHBJR1IY6,4,great taste,This apple butter has a great taste but the pr...,Title: great taste; Content: This apple butter...,39,"[0.0010518254712224007, -0.02572537027299404, ..."
568408,1291420800,B0018CLWM4,ANKM1RMQ4RKQ6,5,Premium Edge Dry Cat Food for Kitten,My 6 month old male Tuxedo cat likes Premium E...,Title: Premium Edge Dry Cat Food for Kitten; C...,170,"[0.014206543564796448, -0.030593300238251686, ..."
568409,1310515200,B0018CLWM4,AJGOF4W50ZNB4,5,Premium Edge Kitten Food,This is a good food with decent ingredients & ...,Title: Premium Edge Kitten Food; Content: This...,82,"[0.0008096409146673977, -0.03716403618454933, ..."


In [18]:
# Load the DataFrame (if not already loaded in memory)
reviews_df["embedding"] = reviews_df["ada_embedding"].apply(np.array)

# Step 1: Create FAISS index
embedding_dim = len(reviews_df["embedding"].iloc[0])
index = faiss.IndexFlatL2(embedding_dim)  # or use IndexFlatIP for cosine similarity with normalized vectors

# Step 2: Add embeddings to the index
embeddings = np.vstack(reviews_df["embedding"].values).astype("float32")
faiss.normalize_L2(embeddings)  # normalize vectors to unit length if using cosine similarity
index.add(embeddings)

# Step 3: Define a search function
def search_reviews(query_text, k=5):
    #embedds the querry input to find k similar sources
    query_vec = np.array(get_embedding(query_text), dtype="float32").reshape(1, -1)
    faiss.normalize_L2(query_vec) # normalize vectors to unit length if using cosine similarity

    distances, indices = index.search(query_vec, k)
    results = reviews_df.iloc[indices[0]][["combined", "embedding"]]
    return results


# 🔍 Example query
query = "This food tastes great and the texture is perfect"
results = search_reviews(query)
print(results["combined"].to_string(index=False))

Id
Title: Great; Content: This is the best brand o...
Title: Will not do without; Content: Great for ...
Title: Great Cafe Latte; Content: This product ...
Title: Exotic food; Content: This is always a g...
Title: Great For Fast Gulasch!; Content: Quick ...


In [27]:
# Hitting the LLM response with relevant documents
def generate_response_with_rag(query, k=5, model="claude-3-7-sonnet-20250219"): #default model loaded in
    # Step 1: Retrieve relevant reviews
    relevant_docs = search_reviews(query, k=k) #rag search query
    context = "\n---\n".join(relevant_docs["combined"].tolist())

    print('\n')
    print('Relevant Documents Retreived From RAG search')
    print(context)

    # Step 2: Format the system prompt
    system_prompt = f"""
			You are a helpful assistant that answers user queries based on customer food reviews.

			Use the following context (customer reviews) to answer the question. If the context does not contain the answer, say you don't know.

			Context:
			{context}
			"""

    # Step 3: Call Claude Chat Completion API
    response = client_ant.messages.create(
        model=model,
        system = system_prompt,
        messages=[
            {"role": "user", "content": query},
            ],
        temperature=0.3,
        max_tokens=300 #set according to your needs
    )

    # Extract and return only the TextBlock
    return response.content

# 🔍 Example usage
user_query = "Are customers happy with the food's texture?" #user query
model = "claude-3-7-sonnet-20250219" #modify model version here
answer = generate_response_with_rag(user_query, model=model)

print('\n')
print("Q:", user_query)
print("A:", answer)



Relevant Documents Retreived From RAG search
Title: Quality & affordable food; Content: I was very pleased with the ingredient quality, taste (according to the kitties), price, and reasonable kibble size of this food. I fed this food to approximately 20 different cats and it was well received by the majority, even the extremely picky one. My cats eat a variety of food. They eat homemade raw, freeze dried raw, grain free dry/canned, and non grain-free dry/canned food. My personal, senior picky cat easily vomits with most food...this food was very easy on her sensitive stomach. I would not hesitate to purchase this food again
---
Title: Happy Camper; Content: I bought this to try on two registered Maine Coon kittens, as one is picky and refused all previous dry foods.  I feed wet food several times a day and keep dry food available at all times.  To my surprise, they both love this dry food.  For a lower priced dry food, it has exceptional ingredients.  Fat content is high, but that's 